In [2]:
import pandas as pd 
# data for testing
actions = pd.read_csv('actions.csv')

In [3]:
#Load modules

#don't worry about these two, but for now, download socceraction module so you can run this.
import socceraction.spadl.config as spadlconfig 
from socceraction.spadl.base import SPADLSchema 

from typing import Callable, List, Tuple
import numpy as np  # type: ignore
from pandera.typing import DataFrame, Series
try:
    from scipy.interpolate import interp2d  # type: ignore
except ImportError:
    interp2d = None
#these two seem be fixed
M: int = 12
N: int = 16

In [4]:
l: int = N #length of zones (16)
w: int = M # width of zones (12)
# we are doing 12 x 16 <- this can change

#This can be the exact same for our hockey functions
def _get_cell_indexes(x: Series, y: Series, l: int = N, w: int = M) -> Tuple[Series, Series]:
    xmin = 0
    ymin = 0

    xi = (x - xmin) / spadlconfig.field_length * l #spadlconfig.length is just the max x 
    yj = (y - ymin) / spadlconfig.field_width * w #spadlconfig.width is just the max y 
    xi = xi.astype(int).clip(0, l - 1)
    yj = yj.astype(int).clip(0, w - 1)
    return xi, yj

xi, yj = _get_cell_indexes(actions.start_x, actions.start_y, l, w)
#xi[:10]
#yj[:10]

In [5]:
# This is what the funciton escencially does. It transforms x and y start to fit our new 
# 16*12 field
print(((actions.start_x - 0) / actions.start_x.max() * 16).astype(int).clip(0,16-1)[:5])
print(((actions.start_y - 0) / actions.start_x.max() * 12).astype(int).clip(0,12-1)[:5])

0    8
1    4
2    4
3    3
4    9
Name: start_x, dtype: int32
0    3
1    4
2    3
3    5
4    3
Name: start_y, dtype: int32


In [6]:
xi, yj = _get_cell_indexes(actions.start_x, actions.start_y, l, w)
xi[:5]
#yj[:5]

0    8
1    4
2    4
3    3
4    9
Name: start_x, dtype: int32

In [7]:
# Values of xi and yj will change if we change values of l and w. 
# For example:
xi, yj = _get_cell_indexes(actions.start_x, actions.start_y, l=14, w=10)
xi.head(5)
#yj.head(5)

0    7
1    4
2    4
3    2
4    8
Name: start_x, dtype: int32

In [8]:
# honestly I don't yet understand what is the purpose of this yet but i assume it will make 
# sense eventually. 
# All I know it is a funciton of length, width, transformed x, and transformed y ona  given point

def _get_flat_indexes(x: Series, y: Series, l: int = N, w: int = M) -> Series:
    xi, yj = _get_cell_indexes(x, y, l, w)
    return l * (w - 1 - yj) + xi 
# yj and xi are the transformed x and y for every point 
# we get in get_cell_indexes

_get_flat_indexes(x=actions.start_x, y=actions.start_y, l=16,w=12)[:5]

0    104
1     84
2    100
3     51
4    121
dtype: int32

In [9]:
#one clue: there are 192 unique values which is = 16*12 (our field)
_get_flat_indexes(x=actions.start_x, y=actions.start_y, l=16,w=12).unique().shape
#So I guess it is giving an index values for every square. and assigning each value
# to all points inside a given square

(192,)

In [10]:
array_=_get_flat_indexes(x=actions.start_x, y=actions.start_y, l=16,w=12) 
array_[array_ == array_[0]].shape # for example, 907 points inside square: array_[0]

(907,)

In [11]:
# This is pretty well commented adn explained. 
# So just read the comments and I'll print the example
#this is where we use flat_indexes, btw
def _count(x: Series, y: Series, l: int = N, w: int = M) -> np.ndarray:
    """Count the number of actions occurring in each cell of the grid.
    Parameters
    ----------
    x : pd.Series
        The x-coordinates of the actions.
    y : pd.Series
        The y-coordinates of the actions.
    l : int
        Amount of grid cells in the x-dimension of the grid.
    w : int
        Amount of grid cells in the y-dimension of the grid.

    Returns
    -------
    np.ndarray
        A matrix, denoting the amount of actions occurring in each cell. The
        top-left corner is the origin.
    """
    #first it gets rid of nan values
    x = x[~np.isnan(x) & ~np.isnan(y)]
    y = y[~np.isnan(x) & ~np.isnan(y)]
    #here are the flat indexes smh
    flat_indexes = _get_flat_indexes(x, y, l, w)
    vc = flat_indexes.value_counts(sort=False) #counts number of points in each square
                                               #store it in a vector
    vector = np.zeros(w * l) #creating a matrix of 0's to transform our cv vector
    vector[vc.index] = vc # there you go
    return vector.reshape((w, l)) #wanna make sure to have the shape of our matrix same as
                                  # our field of coure

#Example using data
print(_count(actions.start_x, actions.start_y, l=16, w= 12))

[[ 112.  239.  381.  452.  874.  887. 1087. 1331. 1439. 1359. 1282. 1545.
  1081.  881.  402.  545.]
 [ 145.  181.  365.  415.  557.  648.  928.  879.  908.  848.  783.  865.
   651.  649.  396.  236.]
 [ 123.  309.  478.  601.  759.  755.  886.  880.  870.  770.  660.  833.
   629.  527.  342.  210.]
 [ 193.  309.  629.  710. 1058.  952. 1282. 1257. 1296. 1074.  750.  827.
   608.  511.  287.  232.]
 [ 334.  424.  540.  515.  757.  774. 1096. 1069.  985.  855.  518.  533.
   427.  356.  220.  125.]
 [1257.  667.  618.  524.  805.  838. 1032.  953.  886.  701.  527.  578.
   375.  300.  257.  158.]
 [1224.  758.  650.  544.  819.  822. 1049.  979.  907.  677.  509.  592.
   397.  259.  293.  132.]
 [ 324.  442.  540.  587.  764.  711. 1088. 1086.  944.  796.  508.  530.
   450.  314.  208.  109.]
 [ 168.  315.  613.  691. 1011.  940. 1101. 1190. 1145.  948.  714.  901.
   578.  486.  260.  179.]
 [ 130.  284.  464.  597.  741.  616.  848.  859.  778.  741.  696.  778.
   597.  537.  34

In [12]:
#this function will divide the matrices at the end of the scoring probability function
#Pretty straightforward stuff, he is using the scoring rate per square as probability
def _safe_divide(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return np.divide(a, b, out=np.zeros_like(a), where=b != 0)

#This is where it gets (more) fun
def scoring_prob(actions: DataFrame[SPADLSchema], l: int = N, w: int = M) -> np.ndarray:
    """Compute the probability of scoring when taking a shot for each cell.

    Parameters
    ----------
    actions : pd.DataFrame
        Actions, in SPADL format.
    l : int
        Amount of grid cells in the x-dimension of the grid.
    w : int
        Amount of grid cells in the y-dimension of the grid.

    Returns
    -------
    np.ndarray
        A matrix, denoting the probability of scoring for each cell.
    """
    #first filters for shots and goals 
    shot_actions = actions[(actions.type_name == 'shot')]
    goals = shot_actions[(shot_actions.result_name == 'success')]
    #uses the function we created before to count number of events per square
    #so we have two matrices (representing our field)
    #one is shots, the other is goals, we divide the two to get the goal rate/shoot per square
    shotmatrix = _count(shot_actions.start_x, shot_actions.start_y, l, w)
    goalmatrix = _count(goals.start_x, goals.start_y, l, w)
    return _safe_divide(goalmatrix, shotmatrix)

In [13]:
#it will work as long as the columsn are named correctly:
#start_x, start_y, type_name, and result_name
#so far this is incredebly easy to adapt to our hockey data
print('shape:',scoring_prob(actions, l=16, w=12).shape)
scoring_prob(actions, l=16, w=12)

shape: (12, 16)


array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.25      ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.05      , 0.06521739, 0.        ,
        0.0625    ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 

In [24]:
actions.type_name.unique()

array(['pass', 'dribble', 'throw_in', 'shot', 'bad_touch', 'clearance',
       'foul', 'freekick_short', 'cross', 'goalkick', 'shot_freekick',
       'corner_crossed', 'take_on', 'tackle', 'corner_short',
       'interception', 'keeper_save', 'keeper_claim', 'freekick_crossed',
       'keeper_punch', 'shot_penalty'], dtype=object)

In [19]:
#here is where we are going to start changing the model.
#developer used pass, dribble, and cross as actions that move the ball. 
#But we might have more event types to select

#here he just filters for the 3 actions that move the ball. The output is a Df with all the columns 
def get_move_actions(actions: DataFrame[SPADLSchema]) -> DataFrame[SPADLSchema]:
    """Get all ball-progressing actions.

    These include passes, dribbles and crosses. Take-ons are ignored because
    they typically coincide with dribbles and do not move the ball to
    a different cell.

    Parameters
    ----------
    actions : pd.DataFrame
        Actions, in SPADL format.

    Returns
    -------
    pd.DataFrame
        All ball-progressing actions in the input dataframe.
    """
    return actions[
        (actions.type_name == 'pass')
        | (actions.type_name == 'dribble')
        | (actions.type_name == 'cross')
    ]
get_move_actions(actions).head(5)

,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,type_id,result_id,bodypart_id,action_id,type_name,result_name,bodypart_name
0,7581,ec5ba260-9bd3-4c5a-b7a5-9f9168ea905d,1,1.0,776,3043,52.941176,33.569620,31.764706,37.012658,0,1,0,0,pass,success,foot
1,7581,abb799d1-73d8-4119-b23f-40f79765f8f8,1,2.0,776,3027,31.764706,37.012658,32.647059,32.708861,21,1,0,1,dribble,success,foot
2,7581,9d3fc438-e85d-49fa-8ff4-21cb3bcc54b1,1,4.0,776,3027,32.647059,32.708861,84.705882,19.797468,0,0,0,2,pass,fail,foot
3,7581,5be94fcb-9612-4f00-82b6-3728f4f1b45c,1,7.0,785,5468,20.294118,48.202532,41.470588,40.455696,0,0,1,3,pass,fail,head
4,7581,0370171b-81a7-49e6-809c-3ade48258b5e,1,9.0,776,5527,63.529412,27.544304,59.117647,27.544304,0,1,1,4,pass,success,head


In [20]:
# now he takes the previous filter and filters for successful actions. Straightforward.

def get_successful_move_actions(actions: DataFrame[SPADLSchema]) -> DataFrame[SPADLSchema]:
    """Get all successful ball-progressing actions.

    These include successful passes, dribbles and crosses.

    Parameters
    ----------
    actions : pd.DataFrame
        Actions, in SPADL format.

    Returns
    -------
    pd.DataFrame
        All ball-progressing actions in the input dataframe.
    """
    move_actions = get_move_actions(actions)
    return move_actions[move_actions.result_name == 'success']
get_successful_move_actions(actions).head(5)

,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,type_id,result_id,bodypart_id,action_id,type_name,result_name,bodypart_name
0,7581,ec5ba260-9bd3-4c5a-b7a5-9f9168ea905d,1,1.0,776,3043,52.941176,33.569620,31.764706,37.012658,0,1,0,0,pass,success,foot
1,7581,abb799d1-73d8-4119-b23f-40f79765f8f8,1,2.0,776,3027,31.764706,37.012658,32.647059,32.708861,21,1,0,1,dribble,success,foot
4,7581,0370171b-81a7-49e6-809c-3ade48258b5e,1,9.0,776,5527,63.529412,27.544304,59.117647,27.544304,0,1,1,4,pass,success,head
5,7581,NaN,1,10.0,776,3959,59.117647,27.544304,65.294118,25.822785,21,1,0,5,dribble,success,foot
8,7581,dc59bcaf-27f6-493f-9b01-312461b14d40,1,14.0,776,3959,59.117647,25.822785,67.058824,24.962025,0,1,0,8,pass,success,foot


In [26]:
#pretty much same idea of what we did before with core prob. 
# he creates 2 diff matrices: sum of shoots per square  and sum of moves (dribble or pass)
# for move action he uses one of the formulas we explained before
#then he gets the prob (rate) for each one of the two possibilities: move ro shoot
#by dividing the matrices
# returns two matrices

def action_prob(
    actions: DataFrame[SPADLSchema], l: int = N, w: int = M
) -> Tuple[np.ndarray, np.ndarray]:
    """Compute the probability of taking an action in each cell of the grid.

    The options are: shooting or moving.

    Parameters
    ----------
    actions : pd.DataFrame
        Actions, in SPADL format.
    l : pd.DataFrame
        Amount of grid cells in the x-dimension of the grid.
    w : pd.DataFrame
        Amount of grid cells in the y-dimension of the grid.

    Returns
    -------
    shotmatrix : np.ndarray
        For each cell the probability of choosing to shoot.
    movematrix : np.ndarray
        For each cell the probability of choosing to move.
    """
    move_actions = get_move_actions(actions)
    shot_actions = actions[(actions.type_name == 'shot')]

    movematrix = _count(move_actions.start_x, move_actions.start_y, l, w)
    shotmatrix = _count(shot_actions.start_x, shot_actions.start_y, l, w)
    totalmatrix = movematrix + shotmatrix

    return _safe_divide(shotmatrix, totalmatrix), _safe_divide(movematrix, totalmatrix)

# remove comment to view
#action_prob(actions,l=16,w=12)[0]
#action_prob(actions,l=16,w=12)[1]

# so far we have 3 diff matrices:
# 1- prob of scoaring when shooting per square
# 2- prob of shooting per square
# 3- prob of moving the ball per square

In [27]:
#Last one before we move onto the xT model


def move_transition_matrix(actions: DataFrame[SPADLSchema], l: int = N, w: int = M) -> np.ndarray:
    """Compute the move transition matrix from the given actions.

    This is, when a player chooses to move, the probability that he will
    end up in each of the other cells of the grid successfully.

    Parameters
    ----------
    actions : pd.DataFrame
        Actions, in SPADL format.
    l : int
        Amount of grid cells in the x-dimension of the grid.
    w : int
        Amount of grid cells in the y-dimension of the grid.

    Returns
    -------
    np.ndarray
        The transition matrix.
    """
    move_actions = get_move_actions(actions)

    X = pd.DataFrame()
    X['start_cell'] = _get_flat_indexes(move_actions.start_x, move_actions.start_y, l, w)
    
    #this is where we are going to use the end_x and end_y. 
    #Which we still need to figure out how to do.
    
    X['end_cell'] = _get_flat_indexes(move_actions.end_x, move_actions.end_y, l, w)
    X['result_name'] = move_actions.result_name
    
    # I don't feel like going step by step here. task for another day.
    vc = X.start_cell.value_counts(sort=False)
    start_counts = np.zeros(w * l)
    start_counts[vc.index] = vc
    transition_matrix = np.zeros((w * l, w * l))
    for i in range(0, w * l):
        vc2 = X[((X.start_cell == i) & (X.result_name == 'success'))].end_cell.value_counts(
            sort=False
        )
        transition_matrix[i, vc2.index] = vc2 / start_counts[i]

    return transition_matrix

In [29]:
# 192 * 192 since we are estomating prob of moving from a square to each of the other 
# squares on the field.
# so we have 36,864 diff probabilities
move_transition_matrix(actions, l=16, w=12).shape

(192, 192)

In [33]:
mtm = move_transition_matrix(actions, l=16, w=12)

In [40]:
# so when a player is on square 0, these are the probabilities of each square 
#to be the end point if the play:
# .21 prob of it staying in square 0
# .11 prob of it moving to square 1, and so on...
mtm[0]

array([0.21126761, 0.11267606, 0.07042254, 0.01408451, 0.01408451,
       0.02816901, 0.01408451, 0.01408451, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.05633803, 0.01408451, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.02816901, 0.04225352, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01408451, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01408451, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.01408451,
       0.        , 0.        , 0.        , 0.        , 0.     